<a href="https://colab.research.google.com/github/Srihari-tech2dive/Smart-Trolley/blob/main/smart_trolley.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install flask fpdf python-barcode ollama

In [8]:
import os
os.makedirs('templates', exist_ok=True)
os.makedirs('static/barcodes', exist_ok=True)

with open('templates/index.html', 'w') as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Smart Trolley</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <script src="https://cdnjs.cloudflare.com/ajax/libs/quagga/0.12.1/quagga.min.js"></script>
    <style>
        body { background: #f0f3f7; }
        .flipkart-blue { background: #2874f0; color: white; padding: 20px; }
        #scanner-container { display: none; width: 100%; height: 250px; border: 2px solid #2874f0; border-radius: 10px; margin-bottom: 15px; }
        #aiSuggestionBox { display: none; background: white; color: #333; width: 50%; margin: 5px auto; border-radius: 5px; text-align: left; padding: 10px; box-shadow: 0 4px 10px rgba(0,0,0,0.15); border: 1px solid #ddd; }
        .suggest-item { cursor: pointer; padding: 8px; border-radius: 4px; transition: 0.2s; }
        .suggest-item:hover { background: #eef5ff; color: #2874f0; }
        .cart-card { min-height: 480px; border-radius: 15px; }
    </style>
</head>
<body>

<div class="flipkart-blue text-center">
    <h3 class="fw-bold">Smart Trolley</h3>
    <input type="text" id="aiSearch" class="form-control w-50 d-inline-block mt-2" placeholder="Search products (e.g. 'd' for Dairy Milk)" oninput="aiPredict(this.value)">

    <div id="aiSuggestionBox">
        <div class="small text-muted mb-1">Are you looking for:</div>
        <div id="suggestedBtn" class="suggest-item fw-bold" onclick="addSuggestedProduct()"></div>
    </div>
</div>

<div class="container mt-4">
    <div class="row">
        <div class="col-md-6">
            <div class="card p-4 shadow-sm border-0 mb-4">
                <button id="scanBtn" class="btn btn-primary w-100 p-3 mb-3 fw-bold" onclick="toggleCamera()">📷 SCAN BARCODE</button>
                <div id="scanner-container"></div>
                <div class="input-group">
                    <input type="text" id="manualCode" class="form-control" placeholder="Enter Code (1001-1004)">
                    <button class="btn btn-dark" onclick="sendItem(document.getElementById('manualCode').value)">Add</button>
                </div>
            </div>
        </div>
        <div class="col-md-6">
            <div class="card p-4 shadow-sm cart-card border-0">
                <h5 class="fw-bold mb-3 border-bottom pb-2">Shopping Cart</h5>
                <div id="cartDisplay"></div>
                <div class="mt-auto border-top pt-3">
                    <div class="d-flex justify-content-between h3 fw-bold"><span>Total:</span><span class="text-primary">₹<span id="totalAmt">0</span></span></div>
                    <button class="btn btn-success w-100 p-3 mt-2 fw-bold" onclick="location.href='/payment_page'">PROCEED TO PAY</button>
                </div>
            </div>
        </div>
    </div>
</div>

<script>
    let scanning = false;
    function aiPredict(val) {
        if(val.length < 1) { document.getElementById('aiSuggestionBox').style.display = "none"; return; }
        fetch('/ai_search', { method:'POST', headers:{'Content-Type':'application/json'}, body:JSON.stringify({text:val}) })
        .then(r=>r.json()).then(d=> {
            if(d.suggestion) {
                document.getElementById('suggestedBtn').innerText = d.suggestion;
                document.getElementById('aiSuggestionBox').style.display = "block";
            } else {
                document.getElementById('aiSuggestionBox').style.display = "none";
            }
        });
    }

    function addSuggestedProduct() {
        let name = document.getElementById('suggestedBtn').innerText;
        sendItem(name);
        document.getElementById('aiSearch').value = "";
        document.getElementById('aiSuggestionBox').style.display = "none";
    }

    function toggleCamera() {
        let container = document.getElementById('scanner-container');
        if (!scanning) {
            container.style.display = 'block';
            Quagga.init({ inputStream: { name: "Live", type: "LiveStream", target: container }, decoder: { readers: ["code_128_reader"] } }, (err) => { Quagga.start(); });
            scanning = true;
        } else {
            Quagga.stop(); container.style.display = 'none'; scanning = false;
        }
    }

    Quagga.onDetected((r) => { toggleCamera(); sendItem(r.codeResult.code); });

    function sendItem(code) {
        fetch('/add_item', { method:'POST', headers:{'Content-Type':'application/json'}, body:JSON.stringify({code:code}) })
        .then(r=>r.json()).then(d=> { if(d.success) updateUI(d); });
    }

    function changeQty(code, action) {
        fetch('/update_qty', { method:'POST', headers:{'Content-Type':'application/json'}, body:JSON.stringify({code:code, action:action}) }).then(r=>r.json()).then(d=> updateUI(d));
    }

    function updateUI(d) {
        let display = document.getElementById('cartDisplay'); display.innerHTML = "";
        for (let code in d.cart) {
            let item = d.cart[code];
            display.innerHTML += `<div class="d-flex justify-content-between align-items-center mb-2 p-3 bg-light rounded shadow-sm">
                <span><b>${item.name}</b><br><small class="text-muted">₹${item.price}/unit</small></span>
                <div class="d-flex align-items-center gap-2">
                    <button class="btn btn-sm btn-outline-danger" onclick="changeQty('${code}', 'minus')">-</button>
                    <b>${item.qty}</b>
                    <button class="btn btn-sm btn-outline-success" onclick="changeQty('${code}', 'plus')">+</button>
                </div>
                <b class="text-primary">₹${item.price * item.qty}</b>
            </div>`;
        }
        document.getElementById('totalAmt').innerText = d.total;
    }
</script>
</body>
</html>""")

with open('templates/payment.html', 'w') as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Secure Checkout</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light p-5">
    <div class="container bg-white p-5 rounded shadow" style="max-width: 500px; border-top: 5px solid #2874f0;">
        <h3 class="text-center fw-bold">Payment Gateway</h3>
        <h4 class="text-center text-primary mt-3 mb-4">Total: ₹ {{total}}</h4>
        <div class="mb-3">
            <label class="form-label fw-bold">Select Method:</label>
            <select id="method" class="form-select mb-3">
                <option value="card">Debit Card (43215678 | PIN: 1122)</option>
                <option value="upi">UPI ID (user@okbank)</option>
                <option value="bank">Bank User ID (admin123)</option>
            </select>
            <input type="text" id="info" class="form-control mb-3" placeholder="Enter Details">
            <input type="password" id="pin" class="form-control mb-3" placeholder="PIN (if card)">
        </div>
        <button class="btn btn-primary w-100 p-3 fw-bold shadow-sm" onclick="pay()">PAY SECURELY</button>
        <div id="success" class="d-none text-center mt-4">
            <h5 class="text-success fw-bold">✔ Payment Successful!</h5>
            <a href="/get_bill" class="btn btn-warning mt-2 w-100 fw-bold shadow-sm">DOWNLOAD PDF RECEIPT</a>
        </div>
    </div>
    <script>
        function pay() {
            let m = document.getElementById('method').value;
            let i = document.getElementById('info').value;
            let p = document.getElementById('pin').value;
            fetch('/pay', { method:'POST', headers:{'Content-Type':'application/json'}, body:JSON.stringify({method:m, num:i, pin:p, upi:i, bank:i}) })
            .then(r=>r.json()).then(d=> { if(d.success) document.getElementById('success').classList.remove('d-none'); else alert("Invalid Payment Data!"); });
        }
    </script>
</body>
</html>""")

In [9]:
from flask import Flask, render_template, request, jsonify, send_file
from fpdf import FPDF
import barcode
from barcode.writer import ImageWriter
import os
import ollama

app = Flask(__name__)


products_db = {
    "1001": {"name": "Dairy Milk", "price": 40},
    "1002": {"name": "Coca Cola", "price": 50},
    "1003": {"name": "Maggi Noodles", "price": 20},
    "1004": {"name": "Lay's Chips", "price": 10}
}

cart = {}

def create_barcodes():
    path = 'static/barcodes'
    if not os.path.exists(path): os.makedirs(path)
    for code in products_db:
        BAR = barcode.get_barcode_class('code128')
        img_bar = BAR(code, writer=ImageWriter())
        img_bar.save(f"{path}/{code}")

create_barcodes()

@app.route('/')
def index():
    global cart
    cart = {}
    return render_template('index.html', products=products_db)

@app.route('/payment_page')
def payment_page():
    total = sum(item['price'] * item['qty'] for item in cart.values())
    return render_template('payment.html', total=total)


@app.route('/ai_search', methods=['POST'])
def ai_search():
    text = request.json.get('text').lower()
    product_names = [v['name'] for v in products_db.values()]


    quick_match = [name for name in product_names if name.lower().startswith(text)]
    if quick_match:
        return jsonify({"suggestion": quick_match[0]})


    prompt = f"Product list: {', '.join(product_names)}. User is typing: '{text}'. Predict the correct product. Output ONLY the name."
    try:
        response = ollama.generate(model='tinydolphin', prompt=prompt)
        prediction = response['response'].strip()
        for name in product_names:
            if name.lower() in prediction.lower():
                return jsonify({"suggestion": name})
    except:
        pass
    return jsonify({"suggestion": ""})

@app.route('/add_item', methods=['POST'])
def add_item():
    code = request.json.get('code')
    found_code = None
    for k, v in products_db.items():
        if k == code or v['name'].lower() == code.lower():
            found_code = k
            break

    if found_code:
        if found_code in cart:
            cart[found_code]['qty'] += 1
        else:
            cart[found_code] = {"name": products_db[found_code]['name'], "price": products_db[found_code]['price'], "qty": 1}
        return jsonify({"success": True, "cart": cart, "total": sum(item['price'] * item['qty'] for item in cart.values())})
    return jsonify({"success": False})

@app.route('/update_qty', methods=['POST'])
def update_qty():
    code, action = request.json.get('code'), request.json.get('action')
    if code in cart:
        if action == 'plus': cart[code]['qty'] += 1
        elif action == 'minus':
            cart[code]['qty'] -= 1
            if cart[code]['qty'] <= 0: del cart[code]
    return jsonify({"success": True, "cart": cart, "total": sum(item['price'] * item['qty'] for item in cart.values())})

@app.route('/pay', methods=['POST'])
def pay():
    data = request.json
    if (data.get('num') == "43215678" and data.get('pin') == "1122") or \
       (data.get('upi') == "user@okbank") or (data.get('bank') == "admin123"):
        return jsonify({"success": True})
    return jsonify({"success": False})

@app.route('/get_bill')
def get_bill():
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 18); pdf.cell(200, 10, txt="SMART TROLLEY RECEIPT", ln=True, align='C')
    pdf.set_font("Arial", size=12); pdf.ln(10)
    total = 0
    for code, item in cart.items():
        line_total = item['price'] * item['qty']
        pdf.cell(80, 10, txt=f"{item['name']} (x{item['qty']})")
        pdf.cell(110, 10, txt=f"Rs.{line_total}", ln=True, align='R')
        total += line_total
    pdf.ln(5); pdf.set_font("Arial", 'B', 14); pdf.cell(190, 10, txt=f"GRAND TOTAL: Rs.{total}", ln=True, align='R')
    pdf.output("invoice.pdf")
    return send_file("invoice.pdf", as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
